In [1]:
import numpy as np
import matplotlib.pyplot as plt
import PIL

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
use_cuda = torch.cuda.is_available()

In [2]:
GPU_NUM = 0 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check

# Additional Infos
if device.type == 'cuda':
    print(torch.cuda.get_device_name(GPU_NUM))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')

Current cuda device  0
GeForce GTX 1660 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


C:\Users\Dustin Baek\.conda\envs\myenv\lib\site-packages\torch\cuda\memory.py:344: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [3]:
###################################### hyperparameters
class HParams():
    def __init__(self):
        self.data_location = 'cat.npz'
        self.enc_hidden_size = 256
        self.dec_hidden_size = 512
        self.Nz = 128
        self.M = 20
        self.dropout = 0.9
        self.batch_size = 100
        self.eta_min = 0.01
        self.R = 0.99995
        self.KL_min = 0.2
        self.wKL = 0.5
        self.lr = 0.001
        self.lr_decay = 0.9999
        self.min_lr = 0.00001
        self.grad_clip = 1.
        self.temperature = 0.4
        self.max_seq_length = 200

hp = HParams()

In [4]:
################################# load and prepare data
def max_size(data):
    """larger sequence length in the data set"""
    sizes = [len(seq) for seq in data]
    return max(sizes)

def purify(strokes):
    """removes to small or too long sequences + removes large gaps"""
    data = []
    for seq in strokes:
        if seq.shape[0] <= hp.max_seq_length and seq.shape[0] > 10:
            seq = np.minimum(seq, 1000)
            seq = np.maximum(seq, -1000)
            seq = np.array(seq, dtype=np.float32)
            data.append(seq)
    return data

def calculate_normalizing_scale_factor(strokes):
    """Calculate the normalizing factor explained in appendix of sketch-rnn."""
    data = []
    for i in range(len(strokes)):
        for j in range(len(strokes[i])):
            data.append(strokes[i][j, 0])
            data.append(strokes[i][j, 1])
    data = np.array(data)
    return np.std(data)

def normalize(strokes):
    """Normalize entire dataset (delta_x, delta_y) by the scaling factor."""
    data = []
    scale_factor = calculate_normalizing_scale_factor(strokes)
    for seq in strokes:
        seq[:, 0:2] /= scale_factor
        data.append(seq)
    return data

In [5]:
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

dataset = np.load(hp.data_location, encoding='latin1')

data = dataset['train']
data = purify(data)
data = normalize(data)
Nmax = max_size(data)
# restore np.load for future normal usage
np.load = np_load_old

In [6]:
############################## function to generate a batch:
def make_batch(batch_size):
    batch_idx = np.random.choice(len(data),batch_size)
    batch_sequences = [data[idx] for idx in batch_idx]
    strokes = []
    lengths = []
    indice = 0
    for seq in batch_sequences:
        len_seq = len(seq[:,0])
        new_seq = np.zeros((Nmax,5))
        new_seq[:len_seq,:2] = seq[:,:2]
        new_seq[:len_seq-1,2] = 1-seq[:-1,2]
        new_seq[:len_seq,3] = seq[:,2]
        new_seq[(len_seq-1):,4] = 1
        new_seq[len_seq-1,2:4] = 0
        lengths.append(len(seq[:,0]))
        strokes.append(new_seq)
        indice += 1

    if use_cuda:
        batch = Variable(torch.from_numpy(np.stack(strokes,1)).cuda().float())
    else:
        batch = Variable(torch.from_numpy(np.stack(strokes,1)).float())
    return batch, lengths

In [7]:
################################ adaptive lr
def lr_decay(optimizer):
    """Decay learning rate by a factor of lr_decay"""
    for param_group in optimizer.param_groups:
        if param_group['lr']>hp.min_lr:
            param_group['lr'] *= hp.lr_decay
    return optimizer

In [8]:
################################# encoder and decoder modules
class EncoderRNN(nn.Module):
    def __init__(self):
        super(EncoderRNN, self).__init__()
        # bidirectional lstm:
        self.lstm = nn.LSTM(5, hp.enc_hidden_size, \
            dropout=hp.dropout, bidirectional=True)
        # create mu and sigma from lstm's last output:
        self.fc_mu = nn.Linear(2*hp.enc_hidden_size, hp.Nz)
        self.fc_sigma = nn.Linear(2*hp.enc_hidden_size, hp.Nz)
        # active dropout:
        self.train()

    def forward(self, inputs, batch_size, hidden_cell=None):
        if hidden_cell is None:
            # then must init with zeros
            if use_cuda:
                hidden = torch.zeros(2, batch_size, hp.enc_hidden_size).cuda()
                cell = torch.zeros(2, batch_size, hp.enc_hidden_size).cuda()
            else:
                hidden = torch.zeros(2, batch_size, hp.enc_hidden_size)
                cell = torch.zeros(2, batch_size, hp.enc_hidden_size)
            hidden_cell = (hidden, cell)
        _, (hidden,cell) = self.lstm(inputs.float(), hidden_cell)
        # hidden is (2, batch_size, hidden_size), we want (batch_size, 2*hidden_size):
        hidden_forward, hidden_backward = torch.split(hidden,1,0)
        hidden_cat = torch.cat([hidden_forward.squeeze(0), hidden_backward.squeeze(0)],1)
        # mu and sigma:
        mu = self.fc_mu(hidden_cat)
        sigma_hat = self.fc_sigma(hidden_cat)
        sigma = torch.exp(sigma_hat/2.)
        # N ~ N(0,1)
        z_size = mu.size()
        if use_cuda:
            N = torch.normal(torch.zeros(z_size),torch.ones(z_size)).cuda()
        else:
            N = torch.normal(torch.zeros(z_size),torch.ones(z_size))
        z = mu + sigma*N
        # mu and sigma_hat are needed for LKL loss
        return z, mu, sigma_hat

In [9]:
class DecoderRNN(nn.Module):
    def __init__(self):
        super(DecoderRNN, self).__init__()
        # to init hidden and cell from z:
        self.fc_hc = nn.Linear(hp.Nz, 2*hp.dec_hidden_size)
        # unidirectional lstm:
        self.lstm = nn.LSTM(hp.Nz+5, hp.dec_hidden_size, dropout=hp.dropout)
        # create proba distribution parameters from hiddens:
        self.fc_params = nn.Linear(hp.dec_hidden_size,6*hp.M+3)

    def forward(self, inputs, z, hidden_cell=None):
        if hidden_cell is None:
            # then we must init from z
            hidden,cell = torch.split(F.tanh(self.fc_hc(z)),hp.dec_hidden_size,1)
            hidden_cell = (hidden.unsqueeze(0).contiguous(), cell.unsqueeze(0).contiguous())
        outputs,(hidden,cell) = self.lstm(inputs, hidden_cell)
        # in training we feed the lstm with the whole input in one shot
        # and use all outputs contained in 'outputs', while in generate
        # mode we just feed with the last generated sample:
        if self.training:
            y = self.fc_params(outputs.view(-1, hp.dec_hidden_size))
        else:
            y = self.fc_params(hidden.view(-1, hp.dec_hidden_size))
        # separate pen and mixture params:
        params = torch.split(y,6,1)
        params_mixture = torch.stack(params[:-1]) # trajectory
        params_pen = params[-1] # pen up/down
        # identify mixture params:
        pi,mu_x,mu_y,sigma_x,sigma_y,rho_xy = torch.split(params_mixture,1,2)
        # preprocess params::
        if self.training:
            len_out = Nmax+1
        else:
            len_out = 1
                                   
        pi = F.softmax(pi.transpose(0,1).squeeze()).view(len_out,-1,hp.M)
        sigma_x = torch.exp(sigma_x.transpose(0,1).squeeze()).view(len_out,-1,hp.M)
        sigma_y = torch.exp(sigma_y.transpose(0,1).squeeze()).view(len_out,-1,hp.M)
        rho_xy = torch.tanh(rho_xy.transpose(0,1).squeeze()).view(len_out,-1,hp.M)
        mu_x = mu_x.transpose(0,1).squeeze().contiguous().view(len_out,-1,hp.M)
        mu_y = mu_y.transpose(0,1).squeeze().contiguous().view(len_out,-1,hp.M)
        q = F.softmax(params_pen).view(len_out,-1,3)
        return pi,mu_x,mu_y,sigma_x,sigma_y,rho_xy,q,hidden,cell

In [15]:
class Model():
    def __init__(self):
        if use_cuda:
            self.encoder = EncoderRNN().cuda()
            self.decoder = DecoderRNN().cuda()
        else:
            self.encoder = EncoderRNN()
            self.decoder = DecoderRNN()
        self.encoder_optimizer = optim.Adam(self.encoder.parameters(), hp.lr)
        self.decoder_optimizer = optim.Adam(self.decoder.parameters(), hp.lr)
        self.eta_step = hp.eta_min

    def make_target(self, batch, lengths):
        if use_cuda:
            eos = torch.stack([torch.Tensor([0,0,0,0,1])]*batch.size()[1]).cuda().unsqueeze(0)
        else:
            eos = torch.stack([torch.Tensor([0,0,0,0,1])]*batch.size()[1]).unsqueeze(0)
        batch = torch.cat([batch, eos], 0)
        mask = torch.zeros(Nmax+1, batch.size()[1])
        for indice,length in enumerate(lengths):
            mask[:length,indice] = 1
        if use_cuda:
            mask = mask.cuda()
        dx = torch.stack([batch.data[:,:,0]]*hp.M,2)
        dy = torch.stack([batch.data[:,:,1]]*hp.M,2)
        p1 = batch.data[:,:,2]
        p2 = batch.data[:,:,3]
        p3 = batch.data[:,:,4]
        p = torch.stack([p1,p2,p3],2)
        return mask,dx,dy,p

    def train(self, epoch):
        self.encoder.train()
        self.decoder.train()
        batch, lengths = make_batch(hp.batch_size)
        # encode:
        z, self.mu, self.sigma = self.encoder(batch, hp.batch_size)
        # create start of sequence:
        if use_cuda:
            sos = torch.stack([torch.Tensor([0,0,1,0,0])]*hp.batch_size).cuda().unsqueeze(0)
        else:
            sos = torch.stack([torch.Tensor([0,0,1,0,0])]*hp.batch_size).unsqueeze(0)
        # had sos at the begining of the batch:
        batch_init = torch.cat([sos, batch],0)
        # expend z to be ready to concatenate with inputs:
        z_stack = torch.stack([z]*(Nmax+1))
        # inputs is concatenation of z and batch_inputs
        inputs = torch.cat([batch_init, z_stack],2)
        # decode:
        self.pi, self.mu_x, self.mu_y, self.sigma_x, self.sigma_y, \
            self.rho_xy, self.q, _, _ = self.decoder(inputs, z)
        # prepare targets:
        mask,dx,dy,p = self.make_target(batch, lengths)
        # prepare optimizers:
        self.encoder_optimizer.zero_grad()
        self.decoder_optimizer.zero_grad()
        # update eta for LKL:
        self.eta_step = 1-(1-hp.eta_min)*hp.R
        # compute losses:
        LKL = self.kullback_leibler_loss()
        LR = self.reconstruction_loss(mask,dx,dy,p,epoch)
        loss = LR + LKL
        # gradient step
        loss.backward()
        # gradient cliping
        nn.utils.clip_grad_norm(self.encoder.parameters(), hp.grad_clip)
        nn.utils.clip_grad_norm(self.decoder.parameters(), hp.grad_clip)
        # optim step
        self.encoder_optimizer.step()
        self.decoder_optimizer.step()
        # some print and save:
        if epoch%1==0:
#             import pdb; pdb.set_trace()
            print('epoch',epoch,'loss',loss.item(),'LR',LR.item(),'LKL',LKL.item())
#             print('epoch',epoch,'loss',loss.data[0],'LR',LR.data[0],'LKL',LKL.data[0])
            self.encoder_optimizer = lr_decay(self.encoder_optimizer)
            self.decoder_optimizer = lr_decay(self.decoder_optimizer)
        if epoch%100==0:
            #self.save(epoch)
#             self.conditional_generation(epoch)
            self.reconstruct()
    
    def bivariate_normal_pdf(self, dx, dy):
        z_x = ((dx-self.mu_x)/self.sigma_x)**2
        z_y = ((dy-self.mu_y)/self.sigma_y)**2
        z_xy = (dx-self.mu_x)*(dy-self.mu_y)/(self.sigma_x*self.sigma_y)
        z = z_x + z_y -2*self.rho_xy*z_xy
        exp = torch.exp(-z/(2*(1-self.rho_xy**2)))
        norm = 2*np.pi*self.sigma_x*self.sigma_y*torch.sqrt(1-self.rho_xy**2)
        return exp/norm

    def reconstruction_loss(self, mask, dx, dy, p, epoch):
        pdf = self.bivariate_normal_pdf(dx, dy)
        LS = -torch.sum(mask*torch.log(1e-5+torch.sum(self.pi * pdf, 2)))\
            /float(Nmax*hp.batch_size)
        LP = -torch.sum(p*torch.log(self.q))/float(Nmax*hp.batch_size)
        return LS+LP

    def kullback_leibler_loss(self):
        LKL = -0.5*torch.sum(1+self.sigma-self.mu**2-torch.exp(self.sigma))\
            /float(hp.Nz*hp.batch_size)
        if use_cuda:
            KL_min = Variable(torch.Tensor([hp.KL_min]).cuda()).detach()
        else:
            KL_min = Variable(torch.Tensor([hp.KL_min])).detach()
        return hp.wKL*self.eta_step * torch.max(LKL,KL_min)

    def save(self, epoch):
        sel = np.random.rand()
        torch.save(self.encoder.state_dict(), \
            'encoderRNN_sel_%3f_epoch_%d.pth' % (sel,epoch))
        torch.save(self.decoder.state_dict(), \
            'decoderRNN_sel_%3f_epoch_%d.pth' % (sel,epoch))

    def load(self, encoder_name, decoder_name):
        saved_encoder = torch.load(encoder_name)
        saved_decoder = torch.load(decoder_name)
        self.encoder.load_state_dict(saved_encoder)
        self.decoder.load_state_dict(saved_decoder)

    def conditional_generation(self, epoch):
        batch,lengths = make_batch(1)
        # should remove dropouts:
        self.encoder.train(False)
        self.decoder.train(False)
        # encode:
        z, _, _ = self.encoder(batch, 1)
        if use_cuda:
            sos = Variable(torch.Tensor([0,0,1,0,0]).view(1,1,-1).cuda())
        else:
            sos = Variable(torch.Tensor([0,0,1,0,0]).view(1,1,-1))
        s = sos
        seq_x = []
        seq_y = []
        seq_z = []
        hidden_cell = None
        for i in range(Nmax):
            input = torch.cat([s,z.unsqueeze(0)],2)
            # decode:
            self.pi, self.mu_x, self.mu_y, self.sigma_x, self.sigma_y, \
                self.rho_xy, self.q, hidden, cell = \
                    self.decoder(input, z, hidden_cell)
            hidden_cell = (hidden, cell)
            # sample from parameters:
            s, dx, dy, pen_down, eos = self.sample_next_state()
#             s, dx, dy, pen_down, eos = self.sample_next()

            #------
            seq_x.append(dx)
            seq_y.append(dy)
            seq_z.append(pen_down)
            if eos:
                print(i)
                break
        # visualize result:
        x_sample = np.cumsum(seq_x, 0)
        y_sample = np.cumsum(seq_y, 0)
        z_sample = np.array(seq_z)
        sequence = np.stack([x_sample,y_sample,z_sample]).T
        make_image(sequence, epoch)

    def sample_next(self, pi, mu_x, mu_y, sigma_x, sigma_y, rho_xy, q):
        pi, mu_x, mu_y, sigma_x, sigma_y, rho_xy, q =\
            pi[0, 0, :], mu_x[0, 0, :], mu_y[0, 0, :], sigma_x[0,
                                                               0, :], sigma_y[0, 0, :], rho_xy[0, 0, :], q[0, 0, :]
        mu_x, mu_y, sigma_x, sigma_y, rho_xy =\
            mu_x.cpu().numpy(), mu_y.cpu().numpy(), sigma_x.cpu(
            ).numpy(), sigma_y.cpu().numpy(), rho_xy.cpu().numpy()
        M = pi.shape[0]
        # offset
        idx = np.random.choice(M, p=pi.cpu().numpy())
        mean = [mu_x[idx], mu_y[idx]]
        cov = [[sigma_x[idx] * sigma_x[idx], rho_xy[idx] * sigma_x[idx]*sigma_y[idx]],
               [rho_xy[idx] * sigma_x[idx]*sigma_y[idx], sigma_y[idx] * sigma_y[idx]]]
        xy = np.random.multivariate_normal(mean, cov, 1)
        xy = torch.from_numpy(xy).float().to(device)

        # pen
        p = torch.tensor([0, 0, 0], device=device, dtype=torch.float)
        idx = np.random.choice(3, p=q.cpu().numpy())
        p[idx] = 1.0
        p = p.unsqueeze(0)

        return torch.cat([xy, p], dim=1).unsqueeze(0)
    
    def sample_next_state(self):

        def adjust_temp(pi_pdf):
            pi_pdf = np.log(pi_pdf)/hp.temperature
            pi_pdf -= pi_pdf.max()
            pi_pdf = np.exp(pi_pdf)
            pi_pdf /= pi_pdf.sum()
            return pi_pdf

        # get mixture indice:
        pi = self.pi.data[0,0,:].cpu().numpy()
        pi = adjust_temp(pi)
        pi_idx = np.random.choice(hp.M, p=pi)
        # get pen state:
        q = self.q.data[0,0,:].cpu().numpy()
        q = adjust_temp(q)
        q_idx = np.random.choice(3, p=q)
        # get mixture params:
        mu_x = self.mu_x.data[0,0,pi_idx]
        mu_y = self.mu_y.data[0,0,pi_idx]
        sigma_x = self.sigma_x.data[0,0,pi_idx]
        sigma_y = self.sigma_y.data[0,0,pi_idx]
        rho_xy = self.rho_xy.data[0,0,pi_idx]
        
        mean = [mu_x, mu_y]
        sigma_x = sigma_x * np.sqrt(hp.temperature)
        sigma_y = sigma_y * np.sqrt(hp.temperature)
        cov = [[sigma_x * sigma_x, rho_xy * sigma_x * sigma_y],\
            [rho_xy * sigma_x * sigma_y, sigma_y * sigma_y]]
        import pdb; pdb.set_trace
        xy = np.random.multivariate_normal(mean, cov, 1)
        x=xy[0][0] 
        y=xy[0][1] 
#         x,y = sample_bivariate_normal(mu_x,mu_y,sigma_x,sigma_y,rho_xy,greedy=False)
        next_state = torch.zeros(5)
        next_state[0] = x
        next_state[1] = y
        next_state[q_idx+2] = 1
        if use_cuda:
            return Variable(next_state.cuda()).view(1,1,-1),x,y,q_idx==1,q_idx==2
        else:
            return Variable(next_state).view(1,1,-1),x,y,q_idx==1,q_idx==2
#         return 1,2,3,4,5

# def sample_bivariate_normal(mu_x,mu_y,sigma_x,sigma_y,rho_xy, greedy=False):
#     # inputs must be floats
#     if greedy:
#         return mu_x,mu_y
#     mean = [mu_x, mu_y]
#     sigma_x = sigma_x * np.sqrt(hp.temperature)
#     sigma_y = sigma_y * np.sqrt(hp.temperature)
#     cov = [[sigma_x * sigma_x, rho_xy * sigma_x * sigma_y],\
#         [rho_xy * sigma_x * sigma_y, sigma_y * sigma_y]]
#     import pdb; pdb.set_trace
#     x = np.random.multivariate_normal(mean, cov, 1)
#     x = torch.from_numpy(xy).float().to(device)

#     return x[0][0], x[0][1]
    def reconstruct(self, S):
        self.encoder.eval()
        self.decoder.eval()
        with torch.no_grad():
            Nmax = S.shape[0]
            batch_size = S.shape[1]
            s_i = torch.stack(
                [torch.tensor([0, 0, 1, 0, 0], device=device, dtype=torch.float)] * batch_size, dim=0).unsqueeze(0)
            output = s_i  # dummy
            z, _, _ = self.encoder(S)
            h0, c0 = torch.split(torch.tanh(self.decoder.fc_hc(z)),
                                 self.decoder.dec_hidden_size, 1)
            hidden_cell = (h0.unsqueeze(0).contiguous(),
                           c0.unsqueeze(0).contiguous())
            for i in range(Nmax):
                (pi, mu_x, mu_y, sigma_x, sigma_y,
                 rho_xy, q), hidden_cell = self.decoder(s_i, z, hidden_cell)
                s_i = self.sample_next(
                    pi, mu_x, mu_y, sigma_x, sigma_y, rho_xy, q)
                output = torch.cat([output, s_i], dim=0)
                if output[-1, 0, 4] == 1:
                    break

            output = output[1:, :, :]  # remove dummy
            return output

    def sample_next(self, pi, mu_x, mu_y, sigma_x, sigma_y, rho_xy, q):
        pi, mu_x, mu_y, sigma_x, sigma_y, rho_xy, q =\
            pi[0, 0, :], mu_x[0, 0, :], mu_y[0, 0, :], sigma_x[0,
                                                               0, :], sigma_y[0, 0, :], rho_xy[0, 0, :], q[0, 0, :]
        mu_x, mu_y, sigma_x, sigma_y, rho_xy =\
            mu_x.cpu().numpy(), mu_y.cpu().numpy(), sigma_x.cpu(
            ).numpy(), sigma_y.cpu().numpy(), rho_xy.cpu().numpy()
        M = pi.shape[0]
        # offset
        idx = np.random.choice(M, p=pi.cpu().numpy())
        mean = [mu_x[idx], mu_y[idx]]
        cov = [[sigma_x[idx] * sigma_x[idx], rho_xy[idx] * sigma_x[idx]*sigma_y[idx]],
               [rho_xy[idx] * sigma_x[idx]*sigma_y[idx], sigma_y[idx] * sigma_y[idx]]]
        xy = np.random.multivariate_normal(mean, cov, 1)
        xy = torch.from_numpy(xy).float().to(device)

        # pen
        p = torch.tensor([0, 0, 0], device=device, dtype=torch.float)
        idx = np.random.choice(3, p=q.cpu().numpy())
        p[idx] = 1.0
        p = p.unsqueeze(0)

        return torch.cat([xy, p], dim=1).unsqueeze(0)


In [17]:
def make_image(sequence, epoch, name='_output_'):
    """plot drawing with separated strokes"""
    strokes = np.split(sequence, np.where(sequence[:,2]>0)[0]+1)
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    for s in strokes:
        plt.plot(s[:,0],-s[:,1])
    canvas = plt.get_current_fig_manager().canvas
    canvas.draw()
    pil_image = PIL.Image.frombytes('RGB', canvas.get_width_height(),
                 canvas.tostring_rgb())
    name = str(epoch)+name+'.jpg'
    pil_image.save(name,"JPEG")
    plt.close("all")

In [51]:
arrays = [np.random.randn(1, 1) for _ in range(10)]
print(arrays)

[array([[-0.44055982]]), array([[-1.59296364]]), array([[0.5278084]]), array([[0.87765278]]), array([[0.16018935]]), array([[-0.11968494]]), array([[0.64920988]]), array([[0.68651249]]), array([[0.08941199]]), array([[-2.07039278]])]


In [62]:
x=np.random.randn(3,4)
print(x)
y=np.random.randn(3,4)
print(y)
np.stack([x,y])

[[ 0.08773955 -0.75696939  0.00500692 -0.41489343]
 [-2.05989091  0.03713604 -0.8290457   0.50611965]
 [ 0.22814642  0.4164204  -1.5747917  -0.11221588]]
[[-0.0441888   0.55901155 -0.48779465 -0.15579444]
 [-1.24987784  0.80118306  0.30387306 -0.28900327]
 [ 0.20882738  0.11877047 -0.76264632  0.53039523]]


array([[[ 0.08773955, -0.75696939,  0.00500692, -0.41489343],
        [-2.05989091,  0.03713604, -0.8290457 ,  0.50611965],
        [ 0.22814642,  0.4164204 , -1.5747917 , -0.11221588]],

       [[-0.0441888 ,  0.55901155, -0.48779465, -0.15579444],
        [-1.24987784,  0.80118306,  0.30387306, -0.28900327],
        [ 0.20882738,  0.11877047, -0.76264632,  0.53039523]]])

In [44]:
mean = [1, 2]
cov = [[1, 0], [0, 1]]  
x = np.random.multivariate_normal(mean, cov, 1)
print(x)
x = torch.from_numpy(x).float().to(device)
print(x.shape)

p = torch.tensor([0, 0, 0], device=device, dtype=torch.float)
idx = np.random.choice(3, 5)
print(idx)
p[idx] = 1.0
print(p)
p = p.unsqueeze(0)
print(p.shape)
z=torch.cat([x, p], dim=1).unsqueeze(0)
print(z.shape)
# print(x[0])
# print(x.shape)

[[-0.23544071  2.51186793]]
torch.Size([1, 2])
[2 0 0 0 1]
tensor([1., 1., 1.], device='cuda:0')
torch.Size([1, 3])
torch.Size([1, 1, 5])


In [13]:
import numpy as np

b = np.zeros(1)
c = np.zeros(1)
c = c/2**63

b += c

In [45]:
model=Model()

C:\Users\Dustin Baek\.conda\envs\myenv\lib\site-packages\torch\nn\modules\rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.9 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [18]:
if __name__=="__main__":
    model = Model()
    for epoch in range(500):
        model.train(epoch)

<ipython-input-9-a948bd699ee2>:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pi = F.softmax(pi.transpose(0,1).squeeze()).view(len_out,-1,hp.M)
<ipython-input-9-a948bd699ee2>:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  q = F.softmax(params_pen).view(len_out,-1,3)
<ipython-input-15-63ddef2dc04f>:66: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.encoder.parameters(), hp.grad_clip)
<ipython-input-15-63ddef2dc04f>:67: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.decoder.parameters(), hp.grad_clip)


epoch 0 loss 2.405754327774048 LR 2.4047493934631348 LKL 0.0010049500269815326
0


C:\Users\Dustin Baek\.conda\envs\myenv\lib\site-packages\torch\nn\functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
<ipython-input-9-a948bd699ee2>:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pi = F.softmax(pi.transpose(0,1).squeeze()).view(len_out,-1,hp.M)
<ipython-input-9-a948bd699ee2>:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  q = F.softmax(params_pen).view(len_out,-1,3)
<ipython-input-15-63ddef2dc04f>:66: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.encoder.parameters(), hp.grad_clip)
<ipython-input-15-63ddef2dc04f>:67: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_gr

epoch 1 loss 2.4273736476898193 LR 2.4263687133789062 LKL 0.0010049500269815326
epoch 2 loss 2.344931125640869 LR 2.343926191329956 LKL 0.0010049500269815326
epoch 3 loss 2.3695778846740723 LR 2.368572950363159 LKL 0.0010049500269815326
epoch 4 loss 2.3616034984588623 LR 2.360598564147949 LKL 0.0010049500269815326
epoch 5 loss 2.240119457244873 LR 2.23911452293396 LKL 0.0010049500269815326
epoch 6 loss 2.035344123840332 LR 2.034339189529419 LKL 0.0010049500269815326
epoch 7 loss 2.218203544616699 LR 2.217198610305786 LKL 0.0010049500269815326
epoch 8 loss 2.0895864963531494 LR 2.0885815620422363 LKL 0.0010049500269815326
epoch 9 loss 1.9057108163833618 LR 1.9047058820724487 LKL 0.0010049500269815326
epoch 10 loss 1.95341157913208 LR 1.952406644821167 LKL 0.0010049500269815326
epoch 11 loss 1.925418496131897 LR 1.9244135618209839 LKL 0.0010049500269815326
epoch 12 loss 1.7686957120895386 LR 1.7676907777786255 LKL 0.0010049500269815326
epoch 13 loss 1.688295841217041 LR 1.687290906906128

C:\Users\Dustin Baek\.conda\envs\myenv\lib\site-packages\torch\nn\functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
<ipython-input-9-a948bd699ee2>:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pi = F.softmax(pi.transpose(0,1).squeeze()).view(len_out,-1,hp.M)
<ipython-input-9-a948bd699ee2>:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  q = F.softmax(params_pen).view(len_out,-1,3)
<ipython-input-15-63ddef2dc04f>:66: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.encoder.parameters(), hp.grad_clip)
<ipython-input-15-63ddef2dc04f>:67: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_gr

...

epoch 201 loss 0.7178159952163696 LR 0.7113367319107056 LKL 0.006479261908680201
epoch 202 loss 0.747210681438446 LR 0.740930438041687 LKL 0.006280216854065657
epoch 203 loss 0.7620486617088318 LR 0.7558327913284302 LKL 0.006215894594788551
epoch 204 loss 0.7004297971725464 LR 0.6943261623382568 LKL 0.0061036208644509315
epoch 205 loss 0.7479207515716553 LR 0.741606593132019 LKL 0.00631413608789444
epoch 206 loss 0.7446556687355042 LR 0.7382557392120361 LKL 0.006399940699338913
epoch 207 loss 0.6589229106903076 LR 0.6521995067596436 LKL 0.006723388563841581
epoch 208 loss 0.7123401165008545 LR 0.7054097056388855 LKL 0.0069303959608078
epoch 209 loss 0.7491775751113892 LR 0.7422913908958435 LKL 0.006886172108352184
epoch 210 loss 0.7133777737617493 LR 0.7062698006629944 LKL 0.00710799265652895
epoch 211 loss 0.7342082858085632 LR 0.727292537689209 LKL 0.006915748585015535
epoch 212 loss 0.6601271033287048 LR 0.6530119180679321 LKL 0.007115173153579235
epoch 213 loss 0.664620041847229 LR

C:\Users\Dustin Baek\.conda\envs\myenv\lib\site-packages\torch\nn\functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
<ipython-input-9-a948bd699ee2>:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pi = F.softmax(pi.transpose(0,1).squeeze()).view(len_out,-1,hp.M)
<ipython-input-9-a948bd699ee2>:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  q = F.softmax(params_pen).view(len_out,-1,3)
<ipython-input-15-63ddef2dc04f>:66: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.encoder.parameters(), hp.grad_clip)
<ipython-input-15-63ddef2dc04f>:67: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_gr

epoch 302 loss 0.5570665001869202 LR 0.5488062500953674 LKL 0.008260239847004414
epoch 303 loss 0.5858306884765625 LR 0.5777614116668701 LKL 0.008069301024079323
epoch 304 loss 0.5674880146980286 LR 0.5594044327735901 LKL 0.008083558641374111
epoch 305 loss 0.5574496388435364 LR 0.5493878126144409 LKL 0.00806181039661169
epoch 306 loss 0.6240548491477966 LR 0.6157549619674683 LKL 0.008299908600747585
epoch 307 loss 0.573070228099823 LR 0.5648547410964966 LKL 0.00821551214903593
epoch 308 loss 0.6161563992500305 LR 0.6077340245246887 LKL 0.008422376587986946
epoch 309 loss 0.5505664944648743 LR 0.5419245362281799 LKL 0.008641960099339485
epoch 310 loss 0.5987535715103149 LR 0.5902228951454163 LKL 0.008530697785317898
epoch 311 loss 0.5731447339057922 LR 0.5644463300704956 LKL 0.008698424324393272
epoch 312 loss 0.5786193609237671 LR 0.5699713826179504 LKL 0.008647997863590717
epoch 313 loss 0.6337071657180786 LR 0.6250166893005371 LKL 0.008690498769283295
epoch 314 loss 0.60002404451370

C:\Users\Dustin Baek\.conda\envs\myenv\lib\site-packages\torch\nn\functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
<ipython-input-9-a948bd699ee2>:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pi = F.softmax(pi.transpose(0,1).squeeze()).view(len_out,-1,hp.M)
<ipython-input-9-a948bd699ee2>:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  q = F.softmax(params_pen).view(len_out,-1,3)
<ipython-input-15-63ddef2dc04f>:66: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.encoder.parameters(), hp.grad_clip)
<ipython-input-15-63ddef2dc04f>:67: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_gr

epoch 401 loss 0.4910261034965515 LR 0.48141375184059143 LKL 0.00961235724389553
epoch 402 loss 0.6134294867515564 LR 0.603863000869751 LKL 0.00956646353006363
epoch 403 loss 0.5109378099441528 LR 0.501406192779541 LKL 0.00953160785138607
epoch 404 loss 0.5978473424911499 LR 0.5882593393325806 LKL 0.009587973356246948
epoch 405 loss 0.445570170879364 LR 0.4360906481742859 LKL 0.00947953574359417
epoch 406 loss 0.5709347724914551 LR 0.5613701939582825 LKL 0.009564594365656376
epoch 407 loss 0.5703397393226624 LR 0.5607577562332153 LKL 0.009581982158124447
epoch 408 loss 0.5142349600791931 LR 0.504578709602356 LKL 0.009656227193772793
epoch 409 loss 0.4452798664569855 LR 0.43544137477874756 LKL 0.009838498197495937
epoch 410 loss 0.47873347997665405 LR 0.4686812162399292 LKL 0.010052253492176533
epoch 411 loss 0.4959622621536255 LR 0.48599353432655334 LKL 0.009968739934265614
epoch 412 loss 0.4812096059322357 LR 0.47117480635643005 LKL 0.010034812614321709
epoch 413 loss 0.43515613675117

In [ ]:
# def sample_bivariate_normal(mu_x,mu_y,sigma_x,sigma_y,rho_xy, greedy=False):
#     # inputs must be floats
#     if greedy:
#         return mu_x,mu_y
#     mean = [mu_x, mu_y]
#     sigma_x = sigma_x * np.sqrt(hp.temperature)
#     sigma_y = sigma_y * np.sqrt(hp.temperature)
#     cov = [[sigma_x * sigma_x, rho_xy * sigma_x * sigma_y],\
#         [rho_xy * sigma_x * sigma_y, sigma_y * sigma_y]]
#     import pdb; pdb.set_trace
#     x = np.random.multivariate_normal(mean, cov, 1)
#     x = torch.from_numpy(xy).float().to(device)

#     return x[0][0], x[0][1]

def make_image(sequence, epoch, name='_output_'):
    """plot drawing with separated strokes"""
    strokes = np.split(sequence, np.where(sequence[:,2]>0)[0]+1)
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    for s in strokes:
        plt.plot(s[:,0],-s[:,1])
    canvas = plt.get_current_fig_manager().canvas
    canvas.draw()
    pil_image = PIL.Image.frombytes('RGB', canvas.get_width_height(),
                 canvas.tostring_rgb())
    name = str(epoch)+name+'.jpg'
    pil_image.save(name,"JPEG")
    plt.close("all")

In [13]:
pi = pi[0, 0, :]
print(pi)

NameError: name 'pi' is not defined